# Make an animated GIF with one Big Data Bowl play

This will allow you to make animated GIFs of any play from the Big Data Bowl frame-by-frame datasets (aka `week1.csv`). You will need to install something like `imagemagick` in order to output to a GIF file. 

In [1]:
!apt-get -qq update
!apt install -y -qq imagemagick 

/bin/bash: apt-get: command not found
The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.



You'll need `pandas` to load the data in, `matplotlib` for the animation function, and `seaborn` for convenient scatter plot creation. You could do this all in `matplotlib` if you wanted to as well. 

In [2]:
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 

# for mpl animation
import matplotlib.animation as animation
from matplotlib import rc
rc('animation', html='html5')

ModuleNotFoundError: No module named 'pandas'

These functions aren't the most cleanly written but they can provide some starting points for those who want to create more interesting animated GIFs as you develop your BDB project. If you install and import the stuff above, the functions below should work. 

In [3]:
def get_play_by_frame(fid, ax, los, one_play):
  """
  take one frame from one play, plot a scatter plot image  

  inputs:
    fid: frame ID  
    ax: current matplotlib ax  
    los: line of scrimmage (for aesthetics)  
    one_play: pandas dataframe for one play  

  output:
    seaborn axis level scatter plot  
  """
  # clear current axis (or else you'll have a tracer effect)
  ax.cla()

  # get game and play IDs
  gid = one_play['gameId'].unique()[0]
  pid = one_play['playId'].unique()[0]

  # isolates a given frame within one play
  one_frame = one_play.loc[one_play['frameId']==fid]

  # create a scatter plot, hard coded dot size to 100 
  fig1 = sns.scatterplot(x='x',y='y',data=one_frame, 
                         hue='team', ax=ax, s=100)
  
  # plots line of scrimmage 
  fig1.axvline(los, c='k', ls=':')

  # plots a simple end zone 
  fig1.axvline(0, c='k', ls='-')
  fig1.axvline(100, c='k', ls='-')

  # game and play IDs as the title
  fig1.set_title(f"game {gid} play {pid}")

  # takes out the legend (if you leave this, you'll get an annoying legend)
  fig1.legend([]).set_visible(False)

  # takes out the left, top, and right borders on the graph 
  sns.despine(left=True)

  # no y axis label
  fig1.set_ylabel('')

  # no y axis tick marks
  fig1.set_yticks([])

  # set the x and y graph limits to the entire field (from kaggle BDB page)
  fig1.set_xlim(-10,110)    
  fig1.set_ylim(0,54) 

def animate_play(one_play):    
  """
  animate a given NFL play from the BDB  

  inputs: 
    one_play: one play from the BDB data. you will want to 
      filter your dataset using gameId and playId.

  output: 
    animated gif, saved to your current working directory 

  """
  # get game and play IDs
  gid = one_play['gameId'].unique()[0]
  pid = one_play['playId'].unique()[0]

  # get line of scrimmage info from the football X location from the  first frame of data 
  los = one_play.loc[(one_play['frameId']==1) & (one_play['team']=='football'), 'x'].values[0]

  # set figure size; this is hard coded but seemed to work well  
  fig = plt.figure(figsize=(14.4, 6.4))

  # get current axis of the figure
  ax = fig.gca()

  # matplotlib animate function
  # relies on get_play_by_frame()  
  # `interval = 100` is something like frames per second i think 
  # repeat=True is to have the animation continuously repeat  
  ani = animation.FuncAnimation(fig, get_play_by_frame, 
                                frames=one_play['frameId'].unique().shape[0],
                                interval=100, repeat=True, 
                                fargs=(ax,los,one_play,))
  
  # close the matplotlib figure when done (if you're batch processing gifs, this allows you to end one gif and begin another gif of a play)
  plt.close()

  # save the matplotlib animation as a gif
  # requires imagemagick or some sort of gif renderer
  # this works in google colab if you apt install imagemagick
  ani.save(f'{gid}_{pid}.gif', writer='imagemagick', fps=10)
  return ani    

In [4]:
# get BDB data. we'll just use one play from week1.csv
data = pd.read_csv('../input/nfl-big-data-bowl-2023/week1.csv')
data.shape

(1118122, 16)

In [5]:
# print the top of the dataframe 
data.head()

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event
0,2021090900,97,25511.0,1,2021-09-10 00:26:31,12.0,TB,right,37.77,24.22,0.29,0.30,0.03,165.16,84.99,None
1,2021090900,97,25511.0,2,2021-09-10 00:26:31,12.0,TB,right,37.78,24.22,0.23,0.11,0.02,164.33,92.87,None
2,2021090900,97,25511.0,3,2021-09-10 00:26:31,12.0,TB,right,37.78,24.24,0.16,0.10,0.01,160.24,68.55,None
3,2021090900,97,25511.0,4,2021-09-10 00:26:31,12.0,TB,right,37.73,24.25,0.15,0.24,0.06,152.13,296.85,None
4,2021090900,97,25511.0,5,2021-09-10 00:26:31,12.0,TB,right,37.69,24.26,0.25,0.18,0.04,148.33,287.55,None


Let's use `gameId = 2021090900` and `playId = 97`

In [6]:
gameId = 2021090900
playId = 97

We can filter our `pandas` dataframe to just this one play with the following conditional filtering:

In [7]:
play = data.loc[(data['gameId']==gameId) & (data['playId']==playId)]
play.shape

(989, 16)

Now that we've filtered our data down to one play, we can put it into our `animate_play()` function  
(note: takes a few seconds to render; if you've been waiting for more than a minute, something might be wrong)

In [8]:
animate_play(play)

<Figure size 432x288 with 0 Axes>

If you run this in the Kaggle notebook this should output into your kaggle working directory. If you run this in the cloud or locally, it will output a GIF to your current working directory. 

# Check out live data science streams at https://twitch.tv/nickwan_datasci 
# Check out data science commentary vids at https://youtube.com/c/NickWan
# If you liked this notebook, please upvote